In [2]:
import pandas as pd
import numpy as np

# DOMA
nodes_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Nodes_processed.csv"
roads_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Roads_processed.csv"
bridges_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\df_bridges_data.csv"
survey_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\surveys\Survey.csv"


# Load the CSV data into a DataFrame

df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')

df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')

df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')

df_survey = pd.read_csv(survey_path, encoding='utf-8', delimiter=',')

C:\Users\Doktor\AppData\Local\Temp\ipykernel_3840\146672732.py:13: DtypeWarning: Columns (5,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
C:\Users\Doktor\AppData\Local\Temp\ipykernel_3840\146672732.py:15: DtypeWarning: Columns (10,11,12,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')


In [3]:
# Assuming df_bridges_all is your DataFrame
columns_to_convert = ['Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška', 'Normálna_zaťažiteľnosť',
                      'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t', 'Počet_otvorov',
                      'Dĺžka_nosnej_konštrukcie_m', 'Dĺžka_premostenia', 'Volná_šírka_mosta',
                      'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2', 'Plocha_mosta_m2']

# Convert specified columns to float
df_bridges_all[columns_to_convert] = df_bridges_all[columns_to_convert].apply(
    pd.to_numeric, errors='coerce')

# Convert Rok_postavenia to integer, assign NaN if not possible
df_bridges_all['Rok_postavenia'] = pd.to_numeric(
    df_bridges_all['Rok_postavenia'], errors='coerce')


df_bridges_all.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Most_DC', 'Názov_mosta', 'ID_DC',
       'správcovské_číslo', 'Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška',
       'Druh_konštrukcie', 'Material', 'Normálna_zaťažiteľnosť',
       'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t',
       'Spôsob_stanovenia', 'Predmet_premostenia', 'Rok_postavenia',
       'Zaťažovacia_trieda_STN', 'Počet_otvorov', 'Dĺžka_premostenia',
       'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
       'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
       'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
       'Smer_úseku', 'Typ_úseku', 'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID',
       'úsekové_staničenie_začiatku_DC_m',
       'Kilometrovníkové_staničenie_začiatku_DC_km',
       'Kumulatívne_staničenie_začiatku_DC_km', 'Poradie', 'Kraj', 'n_2012',
       'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
       'n_2020', 'n_2021', 'n_2022', 'n_2023'],
      

In [5]:
RUSCNR_bridges = df_bridges_all.copy()

# Filter bridges in the Nitra region with Trieda_PK as "cesta I. triedy" or "cesta II. triedy"
RUSCNR_bridges = df_bridges_all[
    ((df_bridges_all['Správca_úseku'] == 'RSUC NR - LV') |
     (df_bridges_all['Správca_úseku'] == 'RSUC NR - NR') |
     (df_bridges_all['Správca_úseku'] == 'RSUC NR - NZ') |
     (df_bridges_all['Správca_úseku'] == 'RSUC NR - TO') |
     (df_bridges_all['Správca_úseku'] == 'RSUC NR - KN')
     )
]

# Reindex the DataFrame
RUSCNR_bridges.reset_index(drop=True, inplace=True)

RUSCNR_bridges = RUSCNR_bridges[RUSCNR_bridges["n_2023"] != 0]

# Display the filtered DataFrame
RUSCNR_bridges.shape

(526, 51)

In [6]:
RUSCNR_bridges["Material"].value_counts()

Material
monolitický železobetón          282
prefabrikovaný železobetón       119
prefabrikovaný predpätý betón     59
oceľ -  plnostenná                32
kameň                             16
tehla                              9
monolitický predpätý betón         4
oceľ -  priehradová                2
oceľobetón                         2
Nezistený                          1
Name: count, dtype: int64

In [34]:
# Filter bridges in the Nitra region with Trieda_PK as "cesta I. triedy" or "cesta II. triedy"
filtered_bridges = RUSCNR_bridges[
    ((RUSCNR_bridges['Material'] == 'prefabrikovaný železobetón') &
     (RUSCNR_bridges['n_2023'] >= 5) &
     (RUSCNR_bridges['Dĺžka_premostenia'] >= 10) 
     )
]

filtered_bridges[["ID_mosta", "Názov_mosta",
                  "Dĺžka_premostenia", "n_2023"]]

,ID_mosta,Názov_mosta,Dĺžka_premostenia,n_2023
136,M6007,Most v katastri Veľký Cetín,18.80,6
194,M2365,Most cez kanál Stará Gúta za Hliníkom.,28.50,5
279,M5823,Most cez hlavný kanál Stará Gúta-Lándor.,20.40,6
280,M7596,Most cez hlavný kanál Stará Gúta-Lándor.,24.70,6
282,M145,Most cez odvodňovací kanál pri Kindeši.,10.15,5
318,M4666,Most cez prívodný kanál k čerpacej stanici v ...,27.55,5
458,M2459,Most v obci Čab-Sila cez potok,18.00,5
459,M5669,Most cez potok Radošinka v obci Malé Ripňany,16.80,6


In [31]:
for index, row in filtered_bridges.iterrows():
    print(row["Názov_mosta"])

Most v katastri Veľký Cetín
Most cez kanál Stará Gúta za Hliníkom.
Most cez hlavný kanál Stará Gúta-Lándor.
Most cez hlavný kanál Stará Gúta-Lándor.
Most cez odvodňovací kanál pri Kindeši.
Most cez prívodný kanál  k čerpacej stanici v obci  Veľké Kosihy
Most v obci Čab-Sila cez potok
Most cez potok Radošinka v obci Malé Ripňany


In [32]:
import geopandas as gpd
# Replace the file path with your actual file path
file_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\GeoJSONs\regions_epsg_4326.geojson'  # doma
# file_path = r'C:\Users\relia\Documents\GitHub\Bridges\GeoJSONs\regions_epsg_4326.geojson'  # robota


# Load the GeoJSON file into a GeoDataFrame
gdf = gpd.read_file(file_path)
# Display the first few rows of the GeoDataFrame

In [33]:
import folium


def plot_filtered_bridges(df, gdf, regions):
    # Find the average coordinates for the initial map center
    avg_lat = df['Zemepisná_šírka'].mean()
    avg_long = df['Zemepisná_dĺžka'].mean()

    # Create the map with custom tileset
    m = folium.Map(location=[avg_lat, avg_long],
                   zoom_start=9, tiles="Cartodb positron")

    # Colors for the markers
    colors = ["#ff6600", "#0066FF"]

    # Bridge icon
    bridge_icon_url = "https://raw.githubusercontent.com/Medvedku/Assets/main/markers/bridge_marker_Cartodb_orng.svg"

    for index, row in df.iterrows():
        ic_size = 50
        bridge_icon = folium.CustomIcon(
            icon_image=bridge_icon_url,
            icon_size=(ic_size, ic_size),
            icon_anchor=(ic_size/2, ic_size)
        )

        popup_html = f"<table style='width:400px;'>"
        popup_html += f"<tr><th style='text-align:left;'>Názov mosta:</th><td>{row['Názov_mosta']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>Trieda PK:</th><td>{row['Trieda_PK']}</td></tr>"

        popup_html += "<tr><th style='text-align:left;'>Dĺžka premostenia:</th><td>"
        if pd.notna(row['Dĺžka_premostenia']):
            popup_html += f"{float(row['Dĺžka_premostenia'])} m"
        else:
            popup_html += "Unknown"
        popup_html += "</td></tr>"

        popup_html += f"<tr><th style='text-align:left;'>Druh konštrukcie:</th><td>{row['Druh_konštrukcie']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>Materiál:</th><td>{row['Material']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>Rok postavenia:</th><td>{row['Rok_postavenia']}</td></tr>"
        popup_html += f"<tr><th style='text-align:left;'>GPS Coordinates:</th><td>({row['Zemepisná_šírka']:.4f}, {row['Zemepisná_dĺžka']:.4f})</td></tr>"
        popup_html += f"</table>"

        popup = folium.Popup(popup_html, parse_html=False)

        tooltip_html = f"<table style='width:50px;'>"
        tooltip_html += f"<tr><th colspan='2' style='text-align:left;'>Most {row['ID_mosta']}</th></tr>"
        tooltip_html += f"<tr><td style='text-align:left;'>STS:</td><td>{row['n_2023']}</td></tr>"
        tooltip_html += f"</table>"

        # Add the marker with the custom icon and popup
        folium.Marker(
            [row['Zemepisná_šírka'], row['Zemepisná_dĺžka']],
            icon=bridge_icon,
            popup=popup,
            tooltip=tooltip_html
        ).add_to(m)

    # Filter the GeoDataFrame to include only the specified regions
    region_gdf = gdf[gdf['NM4'].isin(regions)]

    # Define custom style for the regions' polygons
    def style_function(x): return {
        'fillColor': '#FF6600', 'color': '#FF6600', 'weight': 2}

    # Add the boundaries of the specified regions to the map with the custom style
    folium.GeoJson(region_gdf, name='Regions Boundaries',
                   style_function=style_function).add_to(m)

    return m


# Usage example (replace 'filtered_bridges' with your filtered DataFrame and adjust regions as needed)
regions_to_plot = ['Nitriansky']
map = plot_filtered_bridges(filtered_bridges, gdf, regions_to_plot)
map.save('Bridges_NR.html')